In [5]:
# The purpose of this file is to take colin's data and build a ranking set. 07/05/22
import numpy as np, pickle, os, torch, tqdm, glob
import fingerprint_utils

In [12]:
# using colin's pickle files
pairs_path = "../../tempdata/data/hsqc_ms_pairs"
sets = {}
for f in os.listdir(pairs_path):
    key = f[:f.index(".")] # test, train, or val
    print(f"Key: {key}")
    with open(os.path.join(pairs_path, f), "rb") as f:
        val = pickle.load(f)
    fps = [tuple(v["FP"]) for v in val.values()] # for colin fingerprint
    # fps = [tuple(fingerprint_utils.FP_generator(v["SMILES"], 2)) for v in val.values()] # for hyunwoo fp
    print(f"Original size: {len(fps)}")
    fps_set = set(fps)
    print(f"Size after converting to set: {len(fps_set)}")
    sets[key] = fps_set

Key: train
Original size: 9794
Size after converting to set: 8775
Key: val
Original size: 1224
Size after converting to set: 1204
Key: test
Original size: 1224
Size after converting to set: 1208


In [13]:
super_set = sets["train"].union(sets["val"]).union(sets["test"])
print(f"Number of unique fp's in ranking library: {len(super_set)}, \
    compared to if you added uniques(tr) + uniques(va) + uniques(test): {sum(len(v) for v in sets.values())}")


Number of unique fp's in ranking library: 10759,     compared to if you added uniques(tr) + uniques(va) + uniques(test): 11187


In [9]:
out_dir = "../../tempdata/pair_ranking_set_07_22"
os.makedirs(out_dir, exist_ok=True)

In [10]:
just_test = torch.stack([torch.tensor(v) for v in sets["test"]])
all = torch.stack([torch.tensor(v) for v in super_set])
print(just_test.size())
print(all.size())

torch.Size([1205, 6144])
torch.Size([10663, 6144])


In [11]:
torch.save(just_test, os.path.join(out_dir, "test_pair.pt"))
torch.save(all, os.path.join(out_dir, "all_pair.pt"))